In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json
from tqdm import tqdm
import glob
import math
from typing import List, Tuple
from collections import defaultdict

In [14]:
data_dir = "data"

rooms_data = []
for filename in tqdm(glob.glob(f"{data_dir}/**/*.json", recursive=True), desc="Loading room data"):
    with open(filename, "r") as f:
        data = json.load(f)
        room_name = filename.split("/")[-2]
        data["room_name"] = room_name
        rooms_data.append(data)

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))


Loading room data: 100%|██████████| 17/17 [00:00<00:00, 4679.30it/s]


== SDK package is up to date ==


In [3]:
project = tsdk.get_or_create_project("ertsi_10")

In [4]:
def sort_points_clockwise(points: List[List[float]]) -> List[List[float]]:
    # Compute centroid
    centroid_x = sum(x for x, y in points) / len(points)
    centroid_y = sum(y for x, y in points) / len(points)
    
    # Function to compute angle from centroid
    def angle_from_centroid(point):
        x, y = point
        return math.atan2(y - centroid_y, x - centroid_x)
    
    # Sort points by angle (clockwise)
    return sorted(points, key=angle_from_centroid, reverse=True)

In [15]:
room_definitions = []
for i, room_data in enumerate(rooms_data):
    edge_points_hexagon = sort_points_clockwise(room_data["room_verts"]) if not "sorted" in room_data else room_data["room_verts"]
    room_height = 3
    print(room_data["room_name"], edge_points_hexagon)
    room = treble.GeometryDefinitionGenerator.create_polygon_room(
        points_xy=edge_points_hexagon, height_z=room_height, join_wall_layers=True
    )

    room_definitions.append(room)
room_definitions

markus_08 [[-3.3332183361053467, 2.476045608520508], [3.3332183361053467, 2.476045608520508], [3.3332183361053467, -2.476045608520508], [-3.3332183361053467, -2.476045608520508]]
markus_10 [[-2.0009584426879883, 0.9913800954818726], [-2.0009584426879883, 3.03381609916687], [0.6485803127288818, 3.03381609916687], [0.6485803127288818, 0.9913800954818726], [3.298118829727173, 3.03381609916687], [3.298118829727173, 0.9913800954818726], [3.298118829727173, -1.0510557889938354], [3.298118829727173, -3.03381609916687], [0.6485803127288818, -1.0510557889938354], [0.6485800743103027, -3.03381609916687], [-2.0009584426879883, -3.03381609916687], [-3.298118829727173, -3.03381609916687], [-2.0009584426879883, -1.0510557889938354], [-3.298118829727173, -1.0510557889938354]]
milan_room_1 [[-1.922612190246582, 20.94545555114746], [1.922612190246582, 20.94545555114746], [1.922612190246582, -20.94545555114746], [-1.922612190246582, -20.94545555114746]]
markus_07 [[-8.74837589263916, 12.076228141784668]

[GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.]

In [16]:
generated_rooms = []

print("=== Populating rooms with geometry components ===")
for i, (room_def, room_data) in tqdm(enumerate(zip(room_definitions, rooms_data))):
    print(f"Populating room {room_data["room_name"]} with geometry components")
    room_def.clear_geometry_components()
    placements = []

    if room_data["chair_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
                components=tsdk.geometry_component_library.query(group="chair"),
            preferred_count=room_data["chair_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
    if room_data["desk_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
               components=tsdk.geometry_component_library.query(group="desk"),
            preferred_count=room_data["desk_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
        
    room_def.populate_with_geometry_components(
        components=placements,
        selection_algorithm=treble.ComponentSelectionAlgorithm.random,
    )

    try:
        model = project.add_model(f"room_{i}", room_def)
        generated_rooms.append(model)
    except Exception as e:
        print(f"Error adding model for room {room_data["room_name"]}: {e}")
        continue

=== Populating rooms with geometry components ===


0it [00:00, ?it/s]

Populating room markus_08 with geometry components


1it [00:02,  2.70s/it]

Populating room markus_10 with geometry components


...

15it [00:39,  2.39s/it]

Populating room milan_room_3 with geometry components


16it [00:41,  2.41s/it]

Populating room markus_02 with geometry components


17it [00:45,  2.66s/it]


In [21]:
random_model = random.choice(generated_rooms)
random_model.plot()

In [22]:
dd.display(generated_rooms)

                                                 Models                                                  
┏━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index ┃ Name    ┃ Id                                   ┃ Created At                   ┃ Is Watertight ┃
┡━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0     │ room_0  │ 402019a4-1a3c-4f54-8d91-aecdb7f0fd33 │ 2025-05-28 13:02:44          │ True          │
│ 1     │ room_1  │ bbc9cbd3-8d2c-4a4e-93d0-f38a49d0464b │ 2025-05-28T13:02:47.2226027Z │ False         │
│ 2     │ room_2  │ c020501a-3a6e-496e-89fa-e9a6bd6e7ece │ 2025-05-28T13:02:51.4163193Z │ False         │
│ 3     │ room_3  │ e67b09a4-17a5-46ce-886b-992978454284 │ 2025-05-28T13:02:55.4883981Z │ False         │
│ 4     │ room_4  │ 45d57d5a-77dc-4aec-ad0d-f1d61c5adf6c │ 2025-05-28T13:02:57.9737861Z │ False         │
│ 5     │ room_5  │ 4e34095d-3ce9-43ff-baf9-ed83d263e41f │ 2025-05-28 13:03:00          │ False         │
│ 6     │ room_6  │ 75110543-90ac-4cea-a760-0b26a715fed7 │ 2025-05-28 13:03:02          │ True          │
│ 7     │ room_7  │ e23f3c5b-4730-418b-8cb6-2a480f4710fc │ 2025-05-28T13:03:04.5672069Z │ False         │
│ 8     │ room_8  │ 78904d3b-9bd9-4e43-b8f2-1bc471b0bd13 │ 2025-05-28T13:03:07.2702545Z │ False         │
│ 9     │ room_9  │ 7f3133c6-f36a-4961-bcf6-c38f3e73909d │ 2025-05-28T13:03:09.9298299Z │ False         │
│ 10    │ room_10 │ ef946bcd-627a-41a1-82ba-8c8bed1f472c │ 2025-05-28T13:03:11.9517776Z │ False         │
│ 11    │ room_11 │ 0839419a-3038-4585-8b8d-365d418d1a45 │ 2025-05-28 13:03:14          │ True          │
│ 12    │ room_12 │ 70dfe501-e55d-414a-8c2d-be193dca190e │ 2025-05-28 13:03:17          │ True          │
│ 13    │ room_13 │ d5d1ef28-ace0-46a7-9ee1-f7713cbc2231 │ 2025-05-28T13:03:18.8835238Z │ False         │
│ 14    │ room_14 │ b23852fb-48fa-467b-9bae-496be8c3e791 │ 2025-05-28T13:03:21.4434275Z │ False         │
│ 15    │ room_15 │ 51701c48-dba9-4e00-bf59-0161a5b60061 │ 2025-05-28 13:03:23          │ True          │
│ 16    │ room_16 │ cf32c3be-2f2b-430f-beeb-eb1ccefcb7a7 │ 2025-05-28T13:03:27.1009495Z │ False         │
└───────┴─────────┴──────────────────────────────────────┴──────────────────────────────┴───────────────┘

In [23]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]

all_material_assignments = []
for i, (model, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    layers = {
        "polygon_room_walls": "gypsum/plaster on solid backing",
        "polygon_room_floor": room_data["floor_material"],
        "polygon_room_ceiling": room_data["ceiling_material"],
        "Furniture/Desk": "wood",
        "Furniture/Chair": "upholstered concert chairs",
        "Furniture/Chair A": "upholstered concert chairs",
        "Furniture/Bar Stool": "upholstered concert chairs",
    }
    material_assignment = []

    for layer in model.layer_names:
        if layer in layers:
            search_string = layers[layer]
            matches = [
                m for m in database_materials if search_string.lower() in m.name.lower()
            ]
            if matches:
                material_assignment.append(
                    treble.MaterialAssignment(layer, random.choice(matches))
                )
    all_material_assignments.append(material_assignment)

dd.display(all_material_assignments[-1])

17it [00:01, 12.49it/s]


                                   Material assignment                                   
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Furniture/Chair A    │ Upholstered concert chairs                                     │
│ polygon_room_walls   │ Gypsum/Plaster on solid backing                                │
│ Furniture/Chair      │ Heavily upholstered concert chairs                             │
│ Furniture/Bar Stool  │ Heavily upholstered concert chairs                             │
│ polygon_room_floor   │ Linoleum or vinyl on concrete                                  │
│ Furniture/Desk       │ Wood, 50 mm thick                                              │
│ polygon_room_ceiling │ Slotted panel, 43% open, 40 mm 81 kg/m3 absorber, 30 mm cavity │
└──────────────────────┴────────────────────────────────────────────────────────────────┘

In [24]:
all_sources = []
all_source_positions = []
all_receivers = []
all_receiver_positions = []

pg = treble.PointsGenerator()

pos_ruleset = treble.PointRuleset(
    min_dist_from_surface=0.5,
    min_dist_from_other_points=2,
)

for room in tqdm(generated_rooms):
    pos = pg.generate_valid_points(
        model=room,
        max_count=2,	
        ruleset=pos_ruleset,
        z_range=(0.5, 1.5)
    )
    all_source_positions.append(pos[0])
    all_receiver_positions.append(pos[1])

    source = treble.Source.make_omni(
        position=pos[0],
        label=f"source_{room.name}",
    )
    receiver = treble.Receiver.make_mono(
        position=pos[1],
        label=f"receiver_{room.name}",
    )
    all_sources.append(source)
    all_receivers.append(receiver)


100%|██████████| 17/17 [00:05<00:00,  3.00it/s]


In [25]:
sim_type = treble.SimulationType.geometrical
crossover_frequency = 500

sim_defs = []

for i, (room, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    sim_def = treble.SimulationDefinition(
        name=f"simulation_{i}_1",  # unique name of the simulation
        simulation_type=sim_type,  # the type of simulation
        # crossover_frequency=crossover_frequency,  # the frequency at which the simulation switches from wavebased to image source
        model=room,
        energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -60 dB of energy decay
        receiver_list=[all_receivers[i]],
        source_list=[all_sources[i]],
        material_assignment=all_material_assignments[i],
    )
    sim_defs.append(sim_def)

sim_defs = project.add_simulations(sim_defs)

17it [00:00, 81210.90it/s]


In [26]:
all_sims = project.get_simulations()
dd.display(all_sims)

                                                      Simulations                                                       
┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━┓
┃ Index ┃ Name            ┃ Created at       ┃ Id                                   ┃ Status  ┃ Freq. (Hz) ┃ Rcv ┃ Src ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━┩
│ 0     │ simulation_0_1  │ 2025-05-28 13:06 │ c0de7417-e669-4b66-8ae1-9536be7cae70 │ Created │ 25         │ 1   │ 1   │
│ 1     │ simulation_1_1  │ 2025-05-28 13:06 │ 07ff8966-c67b-4a22-a8b8-c1fe1f210685 │ Created │ 25         │ 1   │ 1   │
│ 2     │ simulation_3_1  │ 2025-05-28 13:06 │ 300d6844-5927-4e6d-8230-6865731d0a1f │ Created │ 25         │ 1   │ 1   │
│ 3     │ simulation_2_1  │ 2025-05-28 13:06 │ 7279daba-6eb7-4716-bec4-e62043aa0a95 │ Created │ 25         │ 1   │ 1   │
│ 4     │ simulation_6_1  │ 2025-05-28 13:06 │ 72a9acf8-cf56-4592-bca8-0f417d950d6a │ Created │ 25         │ 1   │ 1   │
│ 5     │ simulation_5_1  │ 2025-05-28 13:06 │ 8757a1e4-49ac-4e9f-bb8d-8c7a25605c40 │ Created │ 25         │ 1   │ 1   │
│ 6     │ simulation_7_1  │ 2025-05-28 13:06 │ 920ff9ad-13bf-43b7-95ef-368edc5c1400 │ Created │ 25         │ 1   │ 1   │
│ 7     │ simulation_4_1  │ 2025-05-28 13:06 │ b2e8f6c7-5596-40d2-91b1-7fa570921b70 │ Created │ 25         │ 1   │ 1   │
│ 8     │ simulation_10_1 │ 2025-05-28 13:06 │ 296da57b-f945-4df2-b625-3d0e9d06f5ac │ Created │ 25         │ 1   │ 1   │
│ 9     │ simulation_8_1  │ 2025-05-28 13:06 │ 68148b9b-2329-4964-91fc-2d0c57c9e1a8 │ Created │ 25         │ 1   │ 1   │
│ 10    │ simulation_9_1  │ 2025-05-28 13:06 │ 9ff4db82-90a0-4f39-a308-73361c5121ba │ Created │ 25         │ 1   │ 1   │
│ 11    │ simulation_11_1 │ 2025-05-28 13:06 │ a0f1ca48-4fa7-480e-81dc-2c511e99364e │ Created │ 25         │ 1   │ 1   │
│ 12    │ simulation_12_1 │ 2025-05-28 13:06 │ 4488f67d-ae88-4498-badf-a0f0488d662f │ Created │ 25         │ 1   │ 1   │
│ 13    │ simulation_14_1 │ 2025-05-28 13:06 │ 770e5911-311a-48f3-8b90-df5dbfbf3953 │ Created │ 25         │ 1   │ 1   │
│ 14    │ simulation_13_1 │ 2025-05-28 13:06 │ d1cf9cdd-13bb-48ff-a814-ff26877215db │ Created │ 25         │ 1   │ 1   │
│ 15    │ simulation_15_1 │ 2025-05-28 13:06 │ 3602e96b-aa3e-42bd-86eb-9cdda7ad3899 │ Created │ 25         │ 1   │ 1   │
│ 16    │ simulation_16_1 │ 2025-05-28 13:06 │ 5970e5ac-e6c8-4930-bba5-ace69647f904 │ Created │ 25         │ 1   │ 1   │
└───────┴─────────────────┴──────────────────┴──────────────────────────────────────┴─────────┴────────────┴─────┴─────┘

In [27]:
random_sim = random.choice(all_sims)
random_sim.plot()

In [28]:
runtime_estimate = project.estimate()
dd.display(runtime_estimate)


                               Project 2f84d8f4-d59a-4683-a3d2-a1b732a4af85 estimate                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                                                        ┃       Simulation ┃        Estimated ┃   Estimated cost ┃
┃                                             Simulation ┃           status ┃  runtime (hours) ┃         (tokens) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│  simulation_0_1 (c0de7417-e669-4b66-8ae1-9536be7cae70) │          Created │            0.009 │            0.009 │
│  simulation_1_1 (07ff8966-c67b-4a22-a8b8-c1fe1f210685) │          Created │             0.02 │             0.02 │
│  simulation_3_1 (300d6844-5927-4e6d-8230-6865731d0a1f) │          Created │            0.048 │            0.048 │
│  simulation_2_1 (7279daba-6eb7-4716-bec4-e62043aa0a95) │          Created │            0.025 │            0.025 │
│  simulation_6_1 (72a9acf8-cf56-4592-bca8-0f417d950d6a) │          Created │            0.019 │            0.019 │
│  simulation_5_1 (8757a1e4-49ac-4e9f-bb8d-8c7a25605c40) │          Created │            0.022 │            0.022 │
│  simulation_7_1 (920ff9ad-13bf-43b7-95ef-368edc5c1400) │          Created │            0.035 │            0.035 │
│  simulation_4_1 (b2e8f6c7-5596-40d2-91b1-7fa570921b70) │          Created │            0.014 │            0.014 │
│ simulation_10_1 (296da57b-f945-4df2-b625-3d0e9d06f5ac) │          Created │            0.015 │            0.015 │
│  simulation_8_1 (68148b9b-2329-4964-91fc-2d0c57c9e1a8) │          Created │             0.01 │             0.01 │
│  simulation_9_1 (9ff4db82-90a0-4f39-a308-73361c5121ba) │          Created │            0.022 │            0.022 │
│ simulation_11_1 (a0f1ca48-4fa7-480e-81dc-2c511e99364e) │          Created │            0.022 │            0.022 │
│ simulation_12_1 (4488f67d-ae88-4498-badf-a0f0488d662f) │          Created │            0.027 │            0.027 │
│ simulation_14_1 (770e5911-311a-48f3-8b90-df5dbfbf3953) │          Created │            0.042 │            0.042 │
│ simulation_13_1 (d1cf9cdd-13bb-48ff-a814-ff26877215db) │          Created │            0.013 │            0.013 │
│ simulation_15_1 (3602e96b-aa3e-42bd-86eb-9cdda7ad3899) │          Created │            0.014 │            0.014 │
│ simulation_16_1 (5970e5ac-e6c8-4930-bba5-ace69647f904) │          Created │            0.024 │            0.024 │
├────────────────────────────────────────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│                                                  Total │                  │            0.048 │            0.381 │
└────────────────────────────────────────────────────────┴──────────────────┴──────────────────┴──────────────────┘

In [29]:
project.start_simulations()
project.as_live_progress()

                                     ertsi_10                                      
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Simulation Name ┃ Id                                   ┃ Status    ┃ Progress % ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ simulation_1_1  │ 07ff8966-c67b-4a22-a8b8-c1fe1f210685 │ Completed │ 100%       │
│ simulation_10_1 │ 296da57b-f945-4df2-b625-3d0e9d06f5ac │ Completed │ 100%       │
│ simulation_3_1  │ 300d6844-5927-4e6d-8230-6865731d0a1f │ Completed │ 100%       │
│ simulation_15_1 │ 3602e96b-aa3e-42bd-86eb-9cdda7ad3899 │ Completed │ 100%       │
│ simulation_12_1 │ 4488f67d-ae88-4498-badf-a0f0488d662f │ Completed │ 100%       │
│ simulation_16_1 │ 5970e5ac-e6c8-4930-bba5-ace69647f904 │ Completed │ 100%       │
│ simulation_8_1  │ 68148b9b-2329-4964-91fc-2d0c57c9e1a8 │ Completed │ 100%       │
│ simulation_2_1  │ 7279daba-6eb7-4716-bec4-e62043aa0a95 │ Completed │ 100%       │
│ simulation_6_1  │ 72a9acf8-cf56-4592-bca8-0f417d950d6a │ Completed │ 100%       │
│ simulation_14_1 │ 770e5911-311a-48f3-8b90-df5dbfbf3953 │ Completed │ 100%       │
│ simulation_5_1  │ 8757a1e4-49ac-4e9f-bb8d-8c7a25605c40 │ Completed │ 100%       │
│ simulation_7_1  │ 920ff9ad-13bf-43b7-95ef-368edc5c1400 │ Completed │ 100%       │
│ simulation_9_1  │ 9ff4db82-90a0-4f39-a308-73361c5121ba │ Completed │ 100%       │
│ simulation_11_1 │ a0f1ca48-4fa7-480e-81dc-2c511e99364e │ Completed │ 100%       │
│ simulation_4_1  │ b2e8f6c7-5596-40d2-91b1-7fa570921b70 │ Completed │ 100%       │
│ simulation_0_1  │ c0de7417-e669-4b66-8ae1-9536be7cae70 │ Completed │ 100%       │
│ simulation_13_1 │ d1cf9cdd-13bb-48ff-a814-ff26877215db │ Completed │ 100%       │
├─────────────────┼──────────────────────────────────────┼───────────┼────────────┤
│                 │                                      │ Completed │ 100%       │
└─────────────────┴──────────────────────────────────────┴───────────┴────────────┘

Done

In [30]:
for sim, room in zip(all_sims, rooms_data):
    room_name = room["room_name"]
    
    try:
        r = sim.download_results(f'data/{room_name}')
    except ValueError as e:
        print(f"Room {room_name} already exists, skipping download.")
        r = None

/home/leo/projects/repos/ertsi/generation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Download simulation c0de7417-e669-4b66-8ae1-9536be7cae70: 100%|██████████| 2/2 tasks
Download simulation 07ff8966-c67b-4a22-a8b8-c1fe1f210685: 100%|██████████| 2/2 tasks
Download simulation 300d6844-5927-4e6d-8230-6865731d0a1f: 100%|██████████| 2/2 tasks
Download simulation 7279daba-6eb7-4716-bec4-e62043aa0a95: 100%|██████████| 2/2 tasks
Download simulation 72a9acf8-cf56-4592-bca8-0f417d950d6a: 100%|██████████| 2/2 tasks
Download simulation 8757a1e4-49ac-4e9f-bb8d-8c7a25605c40: 100%|██████████| 2/2 tasks
Download simulation 920ff9ad-13bf-43b7-95ef-368edc5c1400: 100%|██████████| 2/2 tasks
Download simulation b2e8f6c7-5596-40d2-91b1-7fa570921b70: 100%|██████████| 2/2 tasks
Download simulation 296da57b-f945-4df2-b625-3d0e9d06f5ac: 100%|██████████| 2/2 tasks
D

In [31]:
for simulation, room in zip(all_sims, rooms_data):
    try:
        results = simulation.get_results_object(f'data/{room["room_name"]}')
        if not results:
            print(f"No results found for room {room['room_name']}")
            continue
        mono_ir = results.get_mono_ir(source=simulation.sources[0], receiver=simulation.receivers[0])
        mono_ir.write_to_wav(path_to_file=f"data/{room['room_name']}/ir.wav")
    except KeyError as e:
        print(f"Results not found for room {room['room_name']}: {e}")
